
### EP2 MAC0417 / MAC5768
##################################################################
#### AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMERO USP,#
#### DECLARO QUE SOU O ÚNICO AUTOR E RESPONSÁVEL PELA RESOLUÇÃO #
#### DESTE EP. #
#### TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM, #
#### SEGUINDO AS INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM #
#### DESONESTIDADE ACADÊMICA OU PLÁGIO. #
# #
#### DECLARO TAMBÉM, QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS #
#### DESSE PROGRAMA, E QUE EU NÃO DISTRIBUI OU FACILITEI A #
#### SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E #
#### DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS #
#### DEFINIDOS NO CÓDIGO DE ÉTICA DA USP. #
#### #
#### ENTENDO QUE JUPYTER NOTEBOOKS SEM ASSINATURA NÃO SERÃO #
#### CORRIGIDOS E, AINDA ASSIM, PODERÃO SER PUNIDOS POR #
#### DESONESTIDADE ACADÊMICA. #
#### #
#### #
#### Nome : Eike Souza da Silva #
#### NUSP : 4618653#
#### Turma:  ? #
#### Prof.: Ronaldo Fumio Hashimoto#
##################################################################


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import cv2
from cv2 import resize

In [2]:
!git clone https://github.com/eike012/mac0417.git

Cloning into 'mac0417'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 77 (delta 4), reused 34 (delta 2), pack-reused 40
Receiving objects: 100% (77/77), 141.69 MiB | 15.52 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [9]:
dir = '/content/mac0417/EP1/imgs'

In [3]:
def df_size(path: str) -> float:
  tot = 0

  for arq in os.listdir(path):
    file_path = os.path.join(path, arq)

    file_size = os.path.getsize(file_path)

    tot += file_size

  in_mb = tot/(1024 * 1024)

  return in_mb

def summary_table(dataframe: object, path: str):

  base_imgs_size = df_size(path)

  class_names = list(set(dataframe['classe']))
  nclass = len(class_names)
  nimgs = len(list(dataframe['nome']))

  img = plt.imread(path + "/1.jpg")
  height, width, _ = img.shape

  print("==============================================")
  print("Tabela Global Sumária: ")
  print(f"  *  ", end="")
  print(f"Nome das classes: ", end="")
  for _class in class_names:
    print(f"{_class}, ", end="")
  print(f" ")
  print(f"  *  Número de classes: {nclass};")
  print(f"  *  Número de imagens: {nimgs};")
  print(f"  *  Tamanho da base (bytes): {round(base_imgs_size)}MB;")
  print(f"  *  Resolução das imagens: {height} linhas por {width} colunas;")
  print("==============================================")

In [4]:
df = pd.read_csv("/content/mac0417/EP1/ep1.csv")

In [5]:
summary_table(df, dir)

Tabela Global Sumária: 
  *  Nome das classes: fiodental, diario, desodorante, livro, oculos, estojo, cremedental,  livro, caneca, fone, guardachuva, caderno, tenis, prestobarba, notebook, controleremoto, hidratante, quadrinho, colher, garrafa1, repelente, celular, joystick, tesoura, garrafa2, bola, carteira,  
  *  Número de classes: 27;
  *  Número de imagens: 36;
  *  Tamanho da base (bytes): 135MB;
  *  Resolução das imagens: 3120 linhas por 4160 colunas;


In [ ]:
for image in os.listdir(dir):
  cam = os.filepath.join(dir, image)
  #print(f'processing {image} . . .')
  img = cv2.imread(cam)
  img_redux = resize(img, (0,0), fx = 0.5, fy = 0.5)

  cv2.imwrite(cam, img_redux)
